In [ ]:
# СТАТИСТИЧЕСКИЙ АНАЛИЗ С DUCKDB (SQL-подобный синтаксис)
print("\n📈 СТАТИСТИЧЕСКИЙ АНАЛИЗ С DUCKDB")

# Создаем DuckDB соединение и загружаем данные
con = duckdb.connect()

# Регистрируем Polars DataFrame в DuckDB
con.register('search_data', df_polars)

# 1. Базовые статистики через SQL
print("1️⃣ ОСНОВНЫЕ МЕТРИКИ:")

basic_stats = con.execute("""
    SELECT 
        platform,
        COUNT(*) as total_queries,
        COUNT(DISTINCT query) as unique_queries,
        AVG(hour) as avg_hour,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hour) as median_hour,
        MIN(hour) as min_hour,
        MAX(hour) as max_hour,
        STDDEV(hour) as std_hour
    FROM search_data
    GROUP BY platform
    ORDER BY platform
""").df()

print(basic_stats)

# 2. Распределение по категориям
print("\n2️⃣ РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:")

category_stats = con.execute("""
    WITH category_counts AS (
        SELECT 
            platform,
            category,
            COUNT(*) as count,
            COUNT(*) * 1.0 / SUM(COUNT(*)) OVER (PARTITION BY platform) as percentage
        FROM search_data
        GROUP BY platform, category
    )
    SELECT 
        platform,
        category,
        count,
        ROUND(percentage * 100, 2) as percentage
    FROM category_counts
    ORDER BY platform, count DESC
""").df()

# Визуализация распределения категорий
fig5 = px.bar(category_stats, 
              x='category', 
              y='percentage',
              color='platform',
              barmode='group',
              title='Распределение запросов по категориям (%)',
              color_discrete_map={'desktop': '#636EFA', 'touch': '#EF553B'},
              labels={'percentage': 'Доля, %', 'category': 'Категория'})

fig5.update_layout(xaxis_title='Категория',
                   yaxis_title='Доля запросов (%)',
                   xaxis_tickangle=-45,
                   height=500)
fig5.show()

# 3. Разница в распределении по часам
print("\n3️⃣ СРАВНЕНИЕ РАСПРЕДЕЛЕНИЙ ПО ЧАСАМ:")

hour_distribution = con.execute("""
    SELECT 
        hour,
        SUM(CASE WHEN platform = 'desktop' THEN 1 ELSE 0 END) as desktop_count,
        SUM(CASE WHEN platform = 'touch' THEN 1 ELSE 0 END) as touch_count,
        desktop_count * 1.0 / SUM(desktop_count) OVER () as desktop_pct,
        touch_count * 1.0 / SUM(touch_count) OVER () as touch_pct
    FROM search_data
    GROUP BY hour
    ORDER BY hour
""").df()

# Визуализация разницы
fig6 = go.Figure()

fig6.add_trace(go.Scatter(
    x=hour_distribution['hour'],
    y=hour_distribution['desktop_pct'] * 100,
    mode='lines+markers',
    name='Desktop',
    line=dict(color='#636EFA', width=3),
    marker=dict(size=8)
))

fig6.add_trace(go.Scatter(
    x=hour_distribution['hour'],
    y=hour_distribution['touch_pct'] * 100,
    mode='lines+markers',
    name='Touch',
    line=dict(color='#EF553B', width=3),
    marker=dict(size=8)
))

fig6.update_layout(
    title='Распределение запросов по часам (нормализованное)',
    xaxis_title='Час суток',
    yaxis_title='Доля запросов (%)',
    xaxis=dict(tickmode='linear', dtick=2),
    height=500,
    hovermode='x unified'
)

fig6.show()